In [1]:
from kinetics_modelling import config

2025-03-26 15:36:12.212 | INFO     | kinetics_modelling.config:<module>:12 - PROJ_ROOT path is: /faststorage/project/mutationalscanning/Workspaces/chcharlton/kinetics_modelling


In [2]:
config.load_config('ob006.ini')['Paths']['bam_filepath']

'~/mutationalscanning/DerivedData/bam/HiFi/human/ob006/kinetics/ob006_kinetics_diploid.bam'

In [8]:
from pathlib import Path
Path(config.load_config('ob006.ini')['Paths']['bam_filepath']).name.split('_')[0]

'ob006'